# Skip Connection

In [1]:
# 라이브러리 준비
import tensorflow as tf

In [2]:
# 데이터 준비
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# 원핫 인코딩 수행
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 10)
(10000, 32, 32, 3) (10000, 10)


In [4]:
# 모델 생성

X = tf.keras.Input([32, 32, 3])
H = tf.keras.layers.Flatten()(X)

H = tf.keras.layers.Dense(256)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation("swish")(H)

for i in range(32):
    H1 = tf.keras.layers.Dense(256)(H)
    H1 = tf.keras.layers.BatchNormalization()(H)
    H1 = tf.keras.layers.Activation("swish")(H)
    H = tf.keras.layers.Add()([H,H1])

Y = tf.keras.layers.Dense(10, activation='softmax')(H)

model = tf.keras.Model(X,Y)
model.compile(loss='categorical_crossentropy', metrics='accuracy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 flatten_1 (Flatten)         (None, 3072)                 0         ['input_2[0][0]']             
                                                                                                  
 dense_2 (Dense)             (None, 256)                  786688    ['flatten_1[0][0]']           
                                                                                                  
 batch_normalization_2 (Bat  (None, 256)                  1024      ['dense_2[0][0]']             
 chNormalization)                                                                             

                                                                     'activation_16[0][0]']       
                                                                                                  
 activation_17 (Activation)  (None, 256)                  0         ['add_13[0][0]']              
                                                                                                  
 add_14 (Add)                (None, 256)                  0         ['add_13[0][0]',              
                                                                     'activation_17[0][0]']       
                                                                                                  
 activation_18 (Activation)  (None, 256)                  0         ['add_14[0][0]']              
                                                                                                  
 add_15 (Add)                (None, 256)                  0         ['add_14[0][0]',              
          

                                                                                                  
 add_30 (Add)                (None, 256)                  0         ['add_29[0][0]',              
                                                                     'activation_33[0][0]']       
                                                                                                  
 activation_34 (Activation)  (None, 256)                  0         ['add_30[0][0]']              
                                                                                                  
 add_31 (Add)                (None, 256)                  0         ['add_30[0][0]',              
                                                                     'activation_34[0][0]']       
                                                                                                  
 dense_35 (Dense)            (None, 10)                   2570      ['add_31[0][0]']              
          

In [ ]:
# 콜백함수 만들기
early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                        min_delta = 0, # 이보다 작게 변하면 변한다고 안할거임.
                                        patience = 10, # 이 회수(epoch)동안 개선이 없으면 끝냄
                                        restore_best_weights=True)
# 모델 학습
result = model.fit(x_train,
                   y_train, 
                   epochs=1000, 
                   batch_size=128, 
                   validation_split=0.2, 
                   callbacks=[early])

Epoch 1/1000
313/313 [==============================] - 11s 27ms/step - loss: 626017728.0000 - accuracy: 0.2890 - val_loss: 1299399424.0000 - val_accuracy: 0.1969
Epoch 2/1000
313/313 [==============================] - 8s 27ms/step - loss: 295146272.0000 - accuracy: 0.3291 - val_loss: 435472288.0000 - val_accuracy: 0.2959
Epoch 3/1000
313/313 [==============================] - 8s 25ms/step - loss: 242622720.0000 - accuracy: 0.3440 - val_loss: 407625600.0000 - val_accuracy: 0.2507
Epoch 4/1000
313/313 [==============================] - 9s 28ms/step - loss: 214909904.0000 - accuracy: 0.3567 - val_loss: 279929696.0000 - val_accuracy: 0.2746
Epoch 5/1000
313/313 [==============================] - 8s 26ms/step - loss: 195688496.0000 - accuracy: 0.3632 - val_loss: 313929696.0000 - val_accuracy: 0.2759
Epoch 6/1000
313/313 [==============================] - 8s 27ms/step - loss: 178526608.0000 - accuracy: 0.3679 - val_loss: 235951072.0000 - val_accuracy: 0.2989
Epoch 7/1000
313/313 [==========

In [ ]:
# 모델 평가하기
model.evaluate(x_test, y_test)

In [ ]:
# 학습 시각화
import matplotlib.pyplot as plt

plt. plot(result.history['loss'])
plt. plot(result.history['val_loss'])
plt.legend(['loss','val_loss'])
plt.show()

plt. plot(result.history['accuracy'])
plt. plot(result.history['val_accuracy'])
plt.legend(['accuracy','val_accuracy'])
plt.show()